In [78]:
from dewloosh.geom import PolyData, Tetra
import pyvista as pv
import numpy as np
import tetgen

In [79]:
# cm
d = 6.0
h = 15.0
a = 15.0

In [80]:
def make_cube(size, shape):
    lx, ly, lz = size
    nx, ny, nz = shape
    x = np.linspace(-lx/2, lx/2, nx)
    y = np.linspace(-ly/2, ly/2, ny)
    z = np.linspace(-lz/2, lz/2, nz)
    grid = pv.StructuredGrid(*np.meshgrid(x, y, z))
    return grid.extract_surface().triangulate()

# Create to examplee PolyData meshes for boolean operations
sphere = pv.Sphere(radius=d/2, center=(0, 0, h/2))
cube = make_cube((a, a, h), (25, 25, 25))
cube.flip_normals()
diff = cube.boolean_difference(sphere)

# tetrahedralize
tet = tetgen.TetGen(diff)
tet.tetrahedralize(order=1, mindihedral=10, minratio=1.1, quality = True)
grid = tet.grid

In [81]:
coords = np.array(grid.points).astype(float)
topo = grid.cells_dict[10].astype(np.int32)
pd = PolyData(coords=coords, topo=topo, celltype=Tetra)

In [82]:
pd.plot()

In [83]:
cyl = pv.CylinderStructured(center=(0.0, 0.0, 0.0), direction=(0.0, 0.0, 1.0),
                            radius=np.linspace(d/2, a/2, 15), height=h, 
                            theta_resolution=100, z_resolution=40)
cyl_surf = cyl.extract_surface().triangulate()
cyl_surf.plot(notebook=False, show_edges=True)